In [105]:
import pandas as pd
import numpy as np

In [106]:
df = pd.read_csv('cleaned_dataset.csv')
df.sort_values('Student Number', inplace=True)
df

,Course Code,Course Title,Type,Course Year,Student Number,Department Code,Course Level,Letter Grade,Status,Semester,Theoritical,Practical,Course Credit,ECTS
1884,CS 201,Data Structures and Algorithms,NaN,2,1,CS,Undergraduate,B+,Successful,2013 - Fall,3,0,3,5
1856,YBS 302,Database Management Systems,NaN,3,1,CS,Undergraduate,A,Successful,2012 - Spring,3,0,3,5
1855,ENGR 498,Global Design Project II Synthesis,NaN,4,1,CS,Undergraduate,A,Successful,2014 - Spring,1,2,2,10
1854,MDB 102,Programming Practice,NaN,1,1,CS,Undergraduate,A,Successful,2012 - Spring,2,2,3,5
1853,CS 362,Machine Intelligence,NaN,3,1,CS,Undergraduate,A,Successful,2013 - Spring,3,0,3,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22872,UNI 123,Textual Analysis and Effective Com.,NaN,1,2773,HIST,Undergraduate,F,Unsuccessful,2012 - Spring,3,0,3,5
22873,UNI 123,Textual Analysis and Effective Com.,NaN,1,2773,HIST,Undergraduate,F,Unsuccessful,2012 - Fall,3,0,3,5
38523,UNI 123,Textual Analysis and Effective Com.,NaN,1,2774,HIST,Undergraduate,F,Unsuccessful,2014 - Fall,3,0,3,5
38524,UNI 201,Formations of Modern Turkey I,NaN,2,2774,HIST,Undergraduate,F,Unsuccessful,2014 - Fall,3,0,3,5


In [107]:
df.insert(7, 'GPA', 0)
df.insert(8, 'Standing', 'Freshman')
df.insert(9, 'Semester of Student', 1)
df.insert(10, 'Completed Credits', 0)
df.insert(11, 'Completed ECTS', 0)
df.insert(12, 'Avg. Grade - Subject', 'B')   # avg grades in courses with that subject
df.insert(13, 'Avg. Grade - Taken', 'B')   # avg grade of students who have taken that course so far
df.insert(14, 'Avg. Grade - Student_Subject', 'B')   # avg. grade of students in courses with the same subject
df.insert(15, 'Repeating Frequency', 0)
df.insert(16, 'Avg. Grade - Prev. Attempts', 'D')

In [108]:
letter_grade_value = {'A+': 4.1, 'A': 4.0, 'A-': 3.7, 'B+': 3.3, 'B': 3.0, 'B-': 2.7, 'C+': 2.3, 'C': 2.0, 'C-': 1.7, 'D+': 1.3, 'D': 1.0, 'D-': 0.5, 'F': 0.0}

In [118]:
def calculate_gpa(df):
    gpa_dict = {}
    completed_credit = {}
    standing_list = ['Freshman', 'Sophomore', 'Junior', 'Senior']
    for row_idx in df.index:
        student_number = df.iloc[row_idx, 4]
        letter_grade = df.iloc[row_idx, 17]
        semester = df.iloc[row_idx, 19]
        credit = df.iloc[row_idx, 22]
        ects = df.iloc[row_idx, 23]
        completed_credit.setdefault(student_number, {})
        completed_credit[student_number].setdefault(semester, [0, 0])
        completed_credit[student_number][semester][0] += credit
        completed_credit[student_number][semester][1] += ects
        gpa_dict.setdefault(student_number, {})
        gpa_dict[student_number].setdefault(semester, [0, 1, 'Freshman'])
        gpa_dict[student_number][semester][0] += letter_grade_value[letter_grade] * credit
    
    for student in gpa_dict:
        sem_counter = 1
        standing_dct = {student: {}}
        standing_counter = 0
        for semester in sorted(gpa_dict[student]):
            year = semester[:4]
            if year not in standing_dct[student] and standing_counter < len(standing_list):
                standing_dct[student][year] = standing_list[standing_counter]
                standing_counter += 1
            gpa_dict[student][semester][1] = sem_counter
            gpa_dict[student][semester][0] /= completed_credit[student][semester][0]
            sem_counter += 1            

        for semester in gpa_dict[student]:
            try:
                gpa_dict[student][semester][2] = standing_dct[student][semester[:4]]
            except KeyError:
                gpa_dict[student][semester][2] = 'Senior'
            
    return gpa_dict, completed_credit

In [119]:
student_gpa, student_credit = calculate_gpa(df)

In [121]:
for row_idx in df.index:
    student_number = df.iloc[row_idx, 4]
    semester = df.iloc[row_idx, 19]
    df.iloc[row_idx, 7] = round(student_gpa[student_number][semester][0], 2)
    df.iloc[row_idx, 8] = student_gpa[student_number][semester][2]
    df.iloc[row_idx, 9] = student_gpa[student_number][semester][1]
    df.iloc[row_idx, 10] = int(student_credit[student_number][semester][0])
    df.iloc[row_idx, 11] = int(student_credit[student_number][semester][1])

In [174]:
def subject_averages(df):
    subject_dict = {}
    
    for idx in df.index:
        semester = df.iloc[idx, 19].strip()
        subject = df.iloc[idx, 0].split()[0].strip()
        grade = df.iloc[idx, 17].strip()
        subject_dict.setdefault(semester, {})
        subject_dict[semester].setdefault(subject, {})
        subject_dict[semester][subject].setdefault(grade, 0)
        subject_dict[semester][subject][grade] += 1

    for semester in subject_dict:
        for subject in subject_dict[semester]:
            sum_ = 0
            count = 0
            for grade in subject_dict[semester][subject]:
                sum_ += letter_grade_value[grade] * subject_dict[semester][subject][grade]
                count += subject_dict[semester][subject][grade]

            average_grade = sum_ / count
            min_ = (10, 'A')
            for letter_grade in letter_grade_value:
                new_min = abs(letter_grade_value[letter_grade] - average_grade)
                if new_min < min_[0]:
                    min_ = (new_min, letter_grade)
                    
            subject_dict[semester][subject] = min_[1]

    return subject_dict       

In [287]:
def avg_grade_student_course(df):
    course_dict = {}
    subject_dict = {}
    df.sort_values('Semester', inplace=True)
    for idx in df.index:
        semester = df.iloc[idx, 19].strip()
        course = df.iloc[idx, 0].strip()
        subject = course.split()[0].strip()
        grade = df.iloc[idx, 17].strip()
        course_dict.setdefault(semester, {})
        subject_dict.setdefault(semester, {})
        course_dict[semester].setdefault(course, {})
        subject_dict[semester].setdefault(subject, {})
        course_dict[semester][course].setdefault(grade, 0)
        subject_dict[semester][subject].setdefault(grade, 0)
        subject_dict[semester][subject][grade] += 1
        course_dict[semester][course][grade] += 1

    for semester in course_dict:
        for course in course_dict[semester]:
            sum_ = 0
            count = 0
            for letter_grade in course_dict[semester][course]:
                sum_ += letter_grade_value[letter_grade] * course_dict[semester][course][letter_grade]
                count += course_dict[semester][course][letter_grade]
            course_dict[semester][course] = (sum_, count)
            
        for subject in subject_dict[semester]:
            sum_ = 0
            count = 0
            for letter_grade in subject_dict[semester][subject]:
                sum_ += letter_grade_value[letter_grade] * subject_dict[semester][subject][letter_grade]
                count += subject_dict[semester][subject][letter_grade]
            subject_dict[semester][subject] = (sum_, count)
    
    averages = {}
    subject_averages = {}
    for semester in sorted(course_dict):
        for course in course_dict[semester]:
            sem_idx = list(sorted(course_dict)).index(semester)
            averages.setdefault(semester, {})
            cum_sum = 0
            cum_count = 0
            for i in range(sem_idx+1):
                if course in course_dict[list(sorted(course_dict))[i]]:
                    cum_sum += course_dict[list(sorted(course_dict))[i]][course][0]
                    cum_count += course_dict[list(sorted(course_dict))[i]][course][1]
            averages[semester][course] = (cum_sum,cum_count)
        
        for subject in subject_dict[semester]:
            sem_idx = list(sorted(subject_dict)).index(semester)
            subject_averages.setdefault(semester, {})
            cum_sum = 0
            cum_count = 0
            for i in range(sem_idx+1):
                if subject in subject_dict[list(sorted(subject_dict))[i]]:
                    cum_sum += subject_dict[list(sorted(subject_dict))[i]][subject][0]
                    cum_count += subject_dict[list(sorted(subject_dict))[i]][subject][1]
            subject_averages[semester][subject] = (cum_sum,cum_count)    
    
    for semester in averages:
        for course in averages[semester]:
            sum_ = 0
            count = 0
            tpl = averages[semester][course]
            sum_ += tpl[0]
            count += tpl[1]

            average_grade = sum_ / count
            min_ = (10, 'A')
            for letter_grade in letter_grade_value:
                new_min = abs(letter_grade_value[letter_grade] - average_grade)
                if new_min < min_[0]:
                    min_ = (new_min, letter_grade)
            course_dict[semester][course] = min_[1]
        
        for subject in subject_averages[semester]:
            sum_ = 0
            count = 0
            tpl = subject_averages[semester][subject]
            sum_ += tpl[0]
            count += tpl[1]
#             print(subject_averages)
            try:
                average_grade = sum_ / count
            except ZeroDivisionError:
                average_grade = 0
            min_ = (10, 'A')
            for letter_grade in letter_grade_value:
                new_min = abs(letter_grade_value[letter_grade] - average_grade)
                if new_min < min_[0]:
                    min_ = (new_min, letter_grade)
            subject_dict[semester][subject] = min_[1]

            
    return course_dict, subject_dict

In [288]:
avg_grade_std, subject_averages = avg_grade_student_course(df)

In [290]:
for row_idx in df.index:
    semester = df.iloc[row_idx, 19]
    course = df.iloc[row_idx, 0].strip()
    subject = course.split()[0].strip()
    df.iloc[row_idx, 13] = avg_grade_std[semester][course]
    df.iloc[row_idx, 14] = subject_averages[semester][subject]
    
    

In [123]:
df.columns

Index(['Course Code', 'Course Title', 'Type', 'Course Year', 'Student Number',
       'Department Code', 'Course Level', 'GPA', 'Standing',
       'Semester of Student', 'Completed Credits', 'Completed ECTS',
       'Avg. Grade - Subject', 'Avg. Grade - Taken',
       'Avg. Grade - Student_Subject', 'Repeating Frequency',
       'Avg. Grade - Prev. Attempts', 'Letter Grade', 'Status', 'Semester',
       'Theoritical', 'Practical', 'Course Credit', 'ECTS'],
      dtype='object')

In [292]:
df.to_csv('extracted_dataset.csv', index=False)